In [ ]:
import selenium
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import lxml
import requests
from lxml import html

import os
import sys
import urllib
import pandas as pd
import cv2

In [2]:
JS_DROP_FILE = """
    var target = arguments[0],
        offsetX = arguments[1],
        offsetY = arguments[2],
        document = target.ownerDocument || document,
        window = document.defaultView || window;

    var input = document.createElement('INPUT');
    input.type = 'file';
    input.onchange = function () {
      var rect = target.getBoundingClientRect(),
          x = rect.left + (offsetX || (rect.width >> 1)),
          y = rect.top + (offsetY || (rect.height >> 1)),
          dataTransfer = { files: this.files };

      ['dragenter', 'dragover', 'drop'].forEach(function (name) {
        var evt = document.createEvent('MouseEvent');
        evt.initMouseEvent(name, !0, !0, window, 0, 0, 0, x, y, !1, !1, !1, !1, 0, null);
        evt.dataTransfer = dataTransfer;
        target.dispatchEvent(evt);
      });

      setTimeout(function () { document.body.removeChild(input); }, 25);
    };
    document.body.appendChild(input);
    return input;
"""

def drag_and_drop_file(drop_target, path):
    driver = drop_target.parent
    file_input = driver.execute_script(JS_DROP_FILE, drop_target, 0, 0)
    file_input.send_keys(path)

In [3]:
def colorScore(img1, img2):
    abs_sum_error=0
    abs_sum_error+=math.fabs(img1.T[0].mean()-img2.T[0].mean())
    abs_sum_error+=math.fabs(img1.T[1].mean()-img2.T[1].mean())
    abs_sum_error+=math.fabs(img1.T[2].mean()-img2.T[2].mean())
    return abs_sum_error

def keyPointScore(img1,img2):
    orb = cv2.ORB_create() 
    queryKeypoints, queryDescriptors = orb.detectAndCompute(img1,None) 
    trainKeypoints, trainDescriptors = orb.detectAndCompute(img2,None) 
    matcher = cv2.BFMatcher() 
    matches = matcher.match(queryDescriptors,trainDescriptors) 
    sums=0.0
    for match in matches:
        sums+=match.distance
    return sums


In [4]:
def getInformation(url:str):
    
    # Getting the webpage, creating a Response object.
    response = requests.get(url)
    
    # Extracting the source code of the page.
    data = response.text
    
    # Passing the source code to BeautifulSoup to create a BeautifulSoup object for it.
    soup = BeautifulSoup(data, 'html.parser')
        
    caption = soup.findAll("div", {"class": "asset-description__caption"})
    tags = soup.findAll("li", {"class": "asset-keywords-list__item"})
    imgur=soup.findAll("img", {"class": "asset-card__image"})
    caption_str=""
    try:
        caption_str:str=caption[0].getText()
    except:
        pass
    tag_list:str=",".join([tag.getText().replace(",","") for tag in tags])
    img_link=imgur[0]["src"]
    dictionary:{}={}
    dictionary["img_url"]=img_link
    dictionary["tags"]=tag_list
    dictionary["caption"]=caption_str
    return  dictionary



In [5]:
def saveImg(dir,original_img, name, urlLink):
        if not os.path.exists(dir):
            os.makedirs(dir) 
            
        subpath:str= os.path.join(dir,original_img)
        if not os.path.exists(subpath):
            os.makedirs(subpath) 
            
        file_name_path:str=os.path.join(subpath,name)+".jpg"
        
        img_url:str=urlLink.replace("?s=2048x2048","") #+".jpg"  

        urllib.request.urlretrieve(img_url, file_name_path)

**RUN COLLECTION ON A SINGLE EXAMPLE**

In [ ]:

driver = webdriver.Chrome('/Users/computer/PycharmProjects/anti_bullying/chromedriver')
driver.get("https://www.gettyimages.co.uk/")
bah=driver.find_element_by_id('btnImageSearch').click()
time.sleep(2)
el=driver.find_element_by_css_selector("[ngf-drop^='dragDropUpload']")
path ='/Users/computer/Downloads/01235.png'

drag_and_drop_file(el,path)

**NEED TO DEAL WITH TIME OUT FOR UPLOAD OPTION**

In [ ]:
topN_images=driver.find_elements_by_class_name("gallery-mosaic-asset__link")

max:int=6

if len(topN_images)<max: max=len(topN_images)

linkInfo=[]

for image in topN_images[:max]:

    linkInfo.append({"url":image.get_attribute("href")})
    
    

In [ ]:


for link in linkInfo:
    
   url=link["url"]
   print(url)
   time.sleep(3)
   link.update(getInformation(url))
   print("next")

**SAVE POSSIBLE IMAGE MATCHES AND THEIR TAGS**

In [ ]:
projectPath="/Users/computer/Downloads/Hateful_Memes_Project/ExampleProject"
elementName="01235.png"#the original png

for counter,link in zip([i for i in range(len(linkInfo))],linkInfo):
    link["src"]=elementName
    link["id"]=str(counter+1)
    time.sleep(2)
    print("saving Image")
    saveImg(projectPath,elementName,link["id"],link["img_url"])

print ("saving Misc Info as a clean csv file")

subpath:str= os.path.join(projectPath,elementName) #directory to save
all_info=pd.DataFrame(linkInfo)
all_info.to_csv(os.path.join(subpath,"related_info.tsv"),sep="\t")

**Find out which is the true image SEE Test_Similarity.ipynb**

In [ ]:
####SEEE Test Similarity Script - this is broken :(
src_img = cv2.imread(original.replace(".png",""), cv2.IMREAD_COLOR)  # trainImage
color_scores=[]
key_point_scores=[]
for img_name in target_imgs:
    img1 = cv2.imread(original.replace(".png","")+"/"+str(img_name)+".jpg", cv2.IMREAD_COLOR)           # queryImage

    img1=cv2.resize(img1,(500,500))
    img2=cv2.resize(src_img,(500,500))
    
    color_score=colorScore(img1,img2)
    key_point_score=keyPointScore(img1,img2)
    color_scores.append(color_score)
    key_point_scores.append(key_point_score)

The following paths are ignored by one of your .gitignore files:
ExampleProject/model-outputs
ExampleProject/submission.csv
Use -f if you really want to add them.
